In [1]:
!apt-get update && apt-get upgrade -y
!apt install -y libgl1-mesa-dev
!apt install -y libglib2.0-0

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 2s (173 kB/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libnvinfer-plugin7 libnvinfer7
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Readin

In [2]:
!pip install opencv-python
!pip install tqdm
!pip install scikit-learn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!nvidia-smi

Mon Dec 19 02:26:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   32C    P8    18W / 340W |     17MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from IPython.display import Image, display

In [5]:
from matplotlib import pyplot as plt

In [6]:
import cv2
from PIL import ImageFile
from PIL import Image

In [7]:
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [9]:
import random
import tqdm
import glob

In [10]:
from datetime import datetime

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
import sys
import pathlib
import os
import shutil
import math
import pickle

In [13]:
import numpy as np

In [14]:
#14 for GeForce RTX 3080 10GB
#55 for V100 40GB
images_per_batch = 14
window_width = 256
window_height = 256
model_name="00/scannoise_remover_model"
era_name="00/scannoise_remover_era"
make_noised_images = "no"
make_image_split = "no"
noise_algorithm = "new"
groundtruthdir="/work/groundtruth/"
cleanimagedir="/work/clean/"
noisedimagedir="/work/noised/"
epochs = 200
eras = 100
images_per_bulk = images_per_batch * 100
validation_percent = 20
restart_era = 5
restart_epoch = 0
testimages_max = 3000
if restart_epoch != 0 or restart_era !=0:
    restart = "yes"
else:
    restart = "no"

In [15]:
def add_linenoise(filename, out_dirname):

    sigma = 0.35
    max_thickness = 20
    min_thickness = 8
    center_high = 24
    center_low = 6

    image_orig = cv2.imread(filename)
    height, width, c = image_orig.shape
    avarage_pixel = int (np.array(image_orig, dtype='float32').sum()/(height*width)/3.)

    output_filename_woext = os.path.splitext(os.path.basename(filename))[0]
    output_dir = os.path.split(os.path.dirname(filename))
    os.makedirs(out_dirname +"/" + output_dir[-1], exist_ok=True)

    counter = 0
    while True:
        randline_img = image_orig.copy()
        randline_img = np.array(randline_img, dtype='float32')
        thickness=random.randint(1,max_thickness/2)*2+1
        rand_x=random.randint(      thickness - int(thickness/2) - 1, width - int(thickness/2) - 1)
        __col = random.randint(0,7)
        _r = __col % 2
        _g = (__col >> 1) % 2
        _b = (__col >> 2) % 2
        _flag = random.randrange(-1,2,2)
        if avarage_pixel > 200 and _flag == 1:
            _flag = -1
        if avarage_pixel < 60 and _flag == -1:
            _flag = 1
        weight_center = random.randrange(center_low,center_high)/100.
        variance_high = int(weight_center*100.)
        variance_low = int(weight_center*100. * 0.85)
        weight_width = variance_high/100.0 - random.randrange(variance_low,variance_high)/100.

        for __x in range(thickness):
            _x = __x - int(thickness/2)
            distance_from_center = - (int(thickness/2) - abs(_x) ) / int(thickness/2) + 1
            gauss_dist = math.exp ( - (distance_from_center) **2  / (2.0 * sigma **2 ) )
            for _y in range(height):
                _col = randline_img[_y, rand_x + _x]
                weight = random.triangular(weight_center-weight_width, weight_center+weight_width, weight_center)
                _col[0] = (_r * 255.) * gauss_dist * _flag * weight + _col [0]
                _col[1] = (_g * 255.) * gauss_dist * _flag * weight + _col [1]
                _col[2] = (_b * 255.) * gauss_dist * _flag * weight + _col [2]
                if _col[0] > 255.:
                    _col[0] = 255.
                if _col[1] > 255.:
                    _col[1] = 255.
                if _col[2] > 255.:
                    _col[2] = 255.
                if _col[0] < 0.:
                    _col[0] = 0.
                if _col[1] < 0.:
                    _col[1] = 0.
                if _col[2] < 0.:
                    _col[2] = 0.
                randline_img[_y, rand_x + _x] = _col
        randline_img = np.array(randline_img, dtype='int16')
        orig_img = np.array(image_orig, dtype='int16')
        randline_img_rgb = randline_img[:,:,0]+randline_img[:,:,1]+randline_img[:,:,2]
        orig_img_rgb = orig_img[:,:,0]+orig_img[:,:,1]+orig_img[:,:,2]

        diff = np.abs(randline_img_rgb - orig_img_rgb)
        diff_max = diff.max()
        diff_sum = diff.sum()
        if diff_sum < 1200 or diff_max < 30:
            counter = counter + 1
            if counter > 10:
                print("diff_max, diff_sum, _flag, weight_center, weight_width, thickness, average_pixel",  output_filename_woext, diff_max, diff_sum, _flag, weight_center, weight_width, thickness, avarage_pixel)
            continue
        else:
            #print("absmax, sum, _flag, weight_center, weight_width, thickness",  output_filename_woext, diff_max, diff_sum, _flag,
            #weight_center, weight_width, thickness, avarage_pixel)
            #print("ok")
            randline_img = np.array(randline_img, dtype='uint8')
            cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext+".png",randline_img)
            return (filename, rand_x, _flag, weight_center, weight_width)

In [16]:
def ImgSplit(filename, out_dirname):
    height = window_height
    width = window_width
    
    im = cv2.imread(filename)
    output_filename_woext = os.path.splitext(os.path.basename(filename))[0]
    output_dir = os.path.split(os.path.dirname(filename))
    os.makedirs(out_dirname +"/" + output_dir[-1], exist_ok=True) 
    img_height, img_width = im.shape[:2]
    # do not make a patch when image patch is too small
    if (img_height < window_height) or (img_width < window_width):
        return
    split_height = int(img_height / height) 
    split_width = int(img_width / width)
    for h1 in range(split_height+1):
        for w1 in range(split_width+1):
            w2 = w1 * width
            h2 = h1 * height
            _h_start = h2
            _h_end   = height+h2-1
            _w_start = w2
            _w_end   = width+w2-1
            if h1 == split_height:
                _h_start = img_height-height
                _h_end   = img_height-1
            if w1 == split_width:
                _w_start = img_width-width
                _w_end   = img_width-1
            c = im[_h_start:_h_end+1, _w_start:_w_end+1]
            if (_h_end+1 - _h_start) == height and (_w_end+1 - _w_start) == width:
                cv2.imwrite(out_dirname +"/" + output_dir[-1] + "/" + output_filename_woext + "_" + str(h1).zfill(3) + "_" + str(w1).zfill(3) +".png",c)


In [17]:
if make_image_split != "no":
    print("making testimages")
    filenames_list=glob.glob(groundtruthdir + "/*/*.jpg")
    try:
        shutil.rmtree(cleanimagedir)
    except OSError as e:
        pass    
    pathlib.Path(cleanimagedir).mkdir()
    
    for _filename in tqdm.tqdm(filenames_list):
        ImgSplit(_filename, out_dirname = cleanimagedir)
else:
    print("do not make testimages")

do not make testimages


In [18]:
if make_noised_images != "no":
    print("making noised images")
    try:
        shutil.rmtree(noisedimagedir)       
    except OSError as e:
        pass
    pathlib.Path(noisedimagedir).mkdir()
    filenames_list=glob.glob(cleanimagedir + "*/*.png")
    for _filename in tqdm.tqdm(filenames_list):
        if noise_algorithm == "old":
            #print("using old noise algorithm")
            pp = add_linenoise_old(_filename, out_dirname = noisedimagedir)
        else:
            #print("using new noise algorithm")
            pp = add_linenoise(_filename, out_dirname = noisedimagedir)

In [19]:
filenames_train_list=sorted(glob.glob(noisedimagedir + "*/*.png"))
filenames_target_list=sorted(glob.glob(cleanimagedir + "*/*.png"))
counter_train = len(filenames_train_list)
counter_target = len(filenames_target_list)
if (counter_train != counter_target):
    print("something wrong in num of pics of target and trains")
    exit(1)
numof_train = int(counter_target * (100.0-validation_percent) / 100.0 )
numof_validation = counter_target - numof_train
print("total images",counter_target)

total images 856074


In [20]:
if restart == "yes":
    print("restarting from era, epoch #", restart_era, restart_epoch)
    era_pickle_file_current=era_name + '.' + str(restart_era).zfill(3) +'.current.pickle'
    era_pickle_file_nextcal=era_name + '.' + str(restart_era).zfill(3) +'.nextcal.pickle'
    with open(era_pickle_file_current, mode='rb') as f:
        ERA_X = pickle.load(f)
    with open(era_pickle_file_nextcal, mode='rb') as f:
        ERA_X_next = pickle.load(f)
    print("current era images: ", len(ERA_X), " restart_era: ", restart_era)
    print("next era images: ",len(ERA_X_next))

else:
    numof_total_files = len(filenames_train_list)
    numof_1st_era_x= numof_total_files-int(counter_target * 0.01)
    ERA_X, ERA_X_next= train_test_split(filenames_train_list, test_size=numof_1st_era_x, random_state=42) #from sklearn

    era_pickle_file_current=era_name + '.' + str(0).zfill(3) +'.current.pickle'
    era_pickle_file_nextcal=era_name + '.' + str(0).zfill(3) +'.nextcal.pickle'

    with open(era_pickle_file_current, mode='wb') as f:
        pickle.dump(ERA_X, f)
    with open(era_pickle_file_nextcal, mode='wb') as f:
        pickle.dump(ERA_X_next, f)

restarting from era, epoch # 5 0
current era images:  45687  restart_era:  5
next era images:  810387


In [21]:
def get_batch(_X_train, _Y_train, _images_per_batch):
    print("getting a batch")
    print("images per batch", _images_per_batch)
    numof_batchs = len(_X_train)//_images_per_batch
    print("Number of batches", numof_batchs)

    i = 0
    while (i < numof_batchs):
        print("doing a batch of ", i, "/", numof_batchs)
        X_batch = []
        Y_batch = []
        Y_batch = _Y_train[(i * _images_per_batch):(i * _images_per_batch + _images_per_batch)]
        X_batch = _X_train[(i * _images_per_batch):(i * _images_per_batch + _images_per_batch)]
        i += 1
        yield X_batch, Y_batch

In [22]:
def get_bulk(_X_train, _Y_train, _images_per_bulk):
    print("images per bluk", _images_per_bulk)
    numof_bulks = len(_X_train)// _images_per_bulk
    print("Number of bulks", numof_bulks)
    i = 0
    while (i < numof_bulks):
        print("doing a bulk of ", i, "/", numof_bulks)
        X_train_bulk = []
        Y_train_bulk = []
        X_train_bulk_name = []
        Y_train_bulk_name = []
        print("getting a bluk...start ")
        X_train_name = _X_train[(i * _images_per_bulk):(i * _images_per_bulk + _images_per_bulk)]
        Y_train_name = _Y_train[(i * _images_per_bulk):(i * _images_per_bulk + _images_per_bulk)]
        X_train_bulk = np.array([np.array(Image.open(file)) for file in X_train_name])
        Y_train_bulk = np.array([np.array(Image.open(file)) for file in Y_train_name])
        print("getting a bluk...done ")

        i += 1
        yield X_train_bulk, Y_train_bulk

In [23]:
# split noised data to training and test (validation) sets
X_train, x_test= train_test_split(ERA_X, test_size=validation_percent, random_state=42) #from sklearn
Y_train = [s.replace(noisedimagedir, cleanimagedir) for s in X_train]
y_test = [s.replace(noisedimagedir, cleanimagedir) for s in x_test]

In [24]:
input_img = Input(shape=(window_height, window_width, 3))
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = Activation('relu')(x)

for i in range(15):
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

x = Conv2D(3, (3, 3), padding='same')(x)
output_img = Activation('tanh')(x)

In [25]:
model = Model(input_img, output_img)
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
import gc

if restart == "yes":
    print("restarting from era, epoch #", restart_era, restart_epoch)
    if restart_epoch != 0:
        load_model(model_name + '.era_' + str(restart_era).zfill(3) + '.epoch_'+ str(restart_epoch).zfill(3) + '.h5')

for era in range(restart_era, eras):
    print("*" * 50)
    print(era, "/", era, " ERAS")
    for epoch in range(restart_epoch, epochs):
        print("=" * 50)
        print(epoch, "/", epochs, " EPOCHS")
        acc = []

        for X_train_bulk, Y_train_bulk in get_bulk(X_train, Y_train, images_per_bulk):
            for X_batch, Y_batch in get_batch(X_train_bulk, Y_train_bulk, images_per_batch):
                print('Training start for a batch')
                # normalize data
                X_batch = X_batch.astype('float32')
                X_batch /= 255
                Y_batch = Y_batch.astype('float32')
                Y_batch /= 255
                model.train_on_batch(X_batch, Y_batch)
                score = model.evaluate(X_batch, Y_batch)
                print("batch accuracy:", score)
                acc.append(score)#
            tf.keras.backend.clear_session()
            gc.collect()
        print("Train accuracy (mean)", np.mean(acc))
        print("Train accuracy (max)", np.max(acc))
        with open(model_name + '.era_' + str(era).zfill(3) + '.epoch_'+ str(epoch).zfill(3) + '.txt', mode='w') as f:
            print("era: ", era, " epoch: ", epoch, " Train accuracy (mean): ", np.mean(acc), " Train accuracy (max): ", np.max(acc), file=f)
        model.save(model_name + '.era_' + str(era).zfill(3) + '.epoch_'+ str(epoch).zfill(3) + '.h5')


restarting from era, epoch # 5 0
**************************************************
5 / 5  ERAS
0 / 200  EPOCHS
images per bluk 1400
Number of bulks 32
doing a bulk of  0 / 32
getting a bluk...start 
getting a bluk...done 
getting a batch
images per batch 14
Number of batches 100
doing a batch of  0 / 100
Training start for a batch
1/1 [==============================] - 0s 289ms/step - loss: 0.6431
batch accuracy: 0.6431452035903931
doing a batch of  1 / 100
Training start for a batch
1/1 [==============================] - 0s 95ms/step - loss: 0.5665
batch accuracy: 0.566451370716095
doing a batch of  2 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.4274
batch accuracy: 0.42738577723503113
doing a batch of  3 / 100
Training start for a batch
1/1 [==============================] - 0s 95ms/step - loss: 0.5814
batch accuracy: 0.5814301371574402
doing a batch of  4 / 100
Training start for a batch
1/1 [==============================] - 0s 94m

1/1 [==============================] - 0s 92ms/step - loss: 0.0329
batch accuracy: 0.03292619809508324
doing a batch of  51 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0280
batch accuracy: 0.028021544218063354
doing a batch of  52 / 100
Training start for a batch
1/1 [==============================] - 0s 94ms/step - loss: 0.0508
batch accuracy: 0.05078047513961792
doing a batch of  53 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0472
batch accuracy: 0.047155823558568954
doing a batch of  54 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0286
batch accuracy: 0.028560152277350426
doing a batch of  55 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0366
batch accuracy: 0.0365958996117115
doing a batch of  56 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - lo

1/1 [==============================] - 0s 90ms/step - loss: 0.0714
batch accuracy: 0.07142140716314316
doing a batch of  2 / 100
Training start for a batch
1/1 [==============================] - 0s 94ms/step - loss: 0.0449
batch accuracy: 0.044893037527799606
doing a batch of  3 / 100
Training start for a batch
1/1 [==============================] - 0s 94ms/step - loss: 0.0505
batch accuracy: 0.05045907944440842
doing a batch of  4 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0525
batch accuracy: 0.05249648913741112
doing a batch of  5 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0477
batch accuracy: 0.04772612079977989
doing a batch of  6 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0498
batch accuracy: 0.04981255531311035
doing a batch of  7 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0

1/1 [==============================] - 0s 92ms/step - loss: 0.0460
batch accuracy: 0.045990023761987686
doing a batch of  54 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.1022
batch accuracy: 0.10223988443613052
doing a batch of  55 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0420
batch accuracy: 0.04197282716631889
doing a batch of  56 / 100
Training start for a batch
1/1 [==============================] - 0s 96ms/step - loss: 0.0590
batch accuracy: 0.05899432674050331
doing a batch of  57 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0346
batch accuracy: 0.03461212292313576
doing a batch of  58 / 100
Training start for a batch
1/1 [==============================] - 0s 94ms/step - loss: 0.0454
batch accuracy: 0.04540777578949928
doing a batch of  59 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - los

1/1 [==============================] - 0s 91ms/step - loss: 0.0361
batch accuracy: 0.03610582277178764
doing a batch of  5 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0570
batch accuracy: 0.05702424794435501
doing a batch of  6 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0385
batch accuracy: 0.03852734714746475
doing a batch of  7 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0249
batch accuracy: 0.024922983720898628
doing a batch of  8 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0473
batch accuracy: 0.0473005585372448
doing a batch of  9 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0341
batch accuracy: 0.03407538682222366
doing a batch of  10 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0

...

1/1 [==============================] - 0s 92ms/step - loss: 0.0076
batch accuracy: 0.007573882583528757
doing a batch of  35 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0074
batch accuracy: 0.007357005029916763
doing a batch of  36 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0053
batch accuracy: 0.005252469331026077
doing a batch of  37 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0127
batch accuracy: 0.012736531905829906
doing a batch of  38 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0060
batch accuracy: 0.0059724715538322926
doing a batch of  39 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0072
batch accuracy: 0.0072059486992657185
doing a batch of  40 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/ste

1/1 [==============================] - 0s 91ms/step - loss: 0.0036
batch accuracy: 0.0036025159060955048
doing a batch of  87 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0038
batch accuracy: 0.003800690174102783
doing a batch of  88 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0027
batch accuracy: 0.002745155245065689
doing a batch of  89 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0032
batch accuracy: 0.0031966606620699167
doing a batch of  90 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0055
batch accuracy: 0.005488971713930368
doing a batch of  91 / 100
Training start for a batch
1/1 [==============================] - 0s 91ms/step - loss: 0.0097
batch accuracy: 0.009706439450383186
doing a batch of  92 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/ste

1/1 [==============================] - 0s 91ms/step - loss: 0.0032
batch accuracy: 0.003241267055273056
doing a batch of  38 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0032
batch accuracy: 0.0031580638606101274
doing a batch of  39 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0053
batch accuracy: 0.005304995458573103
doing a batch of  40 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0051
batch accuracy: 0.005097410641610622
doing a batch of  41 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0059
batch accuracy: 0.005873552523553371
doing a batch of  42 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0034
batch accuracy: 0.0033816799987107515
doing a batch of  43 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/ste

1/1 [==============================] - 0s 95ms/step - loss: 0.0037
batch accuracy: 0.0036555721890181303
doing a batch of  90 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0035
batch accuracy: 0.0035477837081998587
doing a batch of  91 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0035
batch accuracy: 0.0035107338335365057
doing a batch of  92 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0030
batch accuracy: 0.0030260232742875814
doing a batch of  93 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0029
batch accuracy: 0.002916276454925537
doing a batch of  94 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0038
batch accuracy: 0.003803263185545802
doing a batch of  95 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/s

1/1 [==============================] - 0s 93ms/step - loss: 0.0034
batch accuracy: 0.003440575208514929
doing a batch of  41 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0056
batch accuracy: 0.005620626267045736
doing a batch of  42 / 100
Training start for a batch
1/1 [==============================] - 0s 93ms/step - loss: 0.0025
batch accuracy: 0.002463091630488634
doing a batch of  43 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0198
batch accuracy: 0.019845906645059586
doing a batch of  44 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0194
batch accuracy: 0.019362905994057655
doing a batch of  45 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step - loss: 0.0171
batch accuracy: 0.017126720398664474
doing a batch of  46 / 100
Training start for a batch
1/1 [==============================] - 0s 92ms/step 